In [1]:
import pandas as pd
import os
import pickle
import random

In [2]:
from load_datasets import load_asset_ds
from load_datasets import load_automets_ds
from load_datasets import load_benchls_ds
from load_datasets import load_britannica_ds
from load_datasets import load_dwikipedia_ds
from load_datasets import load_ewsewgmpm_ds
from load_datasets import load_ewsewturk_ds
from load_datasets import load_htss_ds
from load_datasets import load_hutssf_ds
from load_datasets import load_massalign_ds
from load_datasets import load_metaeval_ds
from load_datasets import load_mturksf_ds
from load_datasets import load_nnseval_ds
from load_datasets import load_onestopenglish_ds
from load_datasets import load_pwkp_ds
from load_datasets import load_questeval_ds
from load_datasets import load_semeval07_ds
from load_datasets import load_simpa_ds
from load_datasets import load_simpeval_ds
from load_datasets import load_sscorpus_ds
from load_datasets import load_turkcorpus_ds
from load_datasets import load_wikiauto_ds
from load_datasets import load_wikimanual_ds
from load_datasets import load_wikisplit_ds
from load_datasets import load_wikipediav1_ds
from load_datasets import load_wikipediav2_ds
from load_datasets import path_to_datasets

if not os.path.isdir(path_to_datasets):
    os.mkdir(path_to_datasets)

asset = load_asset_ds()
automets = load_automets_ds()
benchls = load_benchls_ds()
britannica = load_britannica_ds()
dwikipedia = load_dwikipedia_ds()
ewsewgmpm = load_ewsewgmpm_ds()
ewsewturk = load_ewsewturk_ds()
htss = load_htss_ds()
hutssf = load_hutssf_ds()
massalign = load_massalign_ds()
metaeval = load_metaeval_ds()
mturksf = load_mturksf_ds()
nnseval = load_nnseval_ds()
onestopenglish = load_onestopenglish_ds()
pwkp = load_pwkp_ds()
questeval = load_questeval_ds()
semeval07 = load_semeval07_ds()
simpa = load_simpa_ds()
simpeval = load_simpeval_ds()
sscorpus = load_sscorpus_ds()
turkcorpus = load_turkcorpus_ds()
wikiauto = load_wikiauto_ds()
wikimanual = load_wikimanual_ds()
wikisplit = load_wikisplit_ds()
wikipediav1 = load_wikipediav1_ds()
wikipediav2 = load_wikipediav2_ds()

combined_dataset = pd.concat([asset, automets, benchls, britannica, dwikipedia, ewsewgmpm, ewsewturk, htss, hutssf, massalign, metaeval, 
                              mturksf, nnseval, onestopenglish, pwkp, questeval, semeval07, simpa, simpeval, sscorpus, turkcorpus, 
                              wikiauto, wikimanual, wikisplit, wikipediav1, wikipediav2], axis=0).reset_index()


In [3]:
text_and_base_ds = []
all_txts = []
counts_per_ds = {}
num_simp_per_ds = {}
num_src_per_ds = {}
only_texts = []

# shuffle combined dataset
combined_shuffled_dataset = combined_dataset.sample(frac=1, random_state = 42).reset_index(drop=True)

In [9]:
max_num = 50

In [4]:
for index, row in combined_shuffled_dataset.iterrows():
    if row['ds_id'] not in num_src_per_ds:
        num_src_per_ds[row['ds_id']] = 0
        num_simp_per_ds[row['ds_id']] = 0
        counts_per_ds[row['ds_id']] = 0

    if row['ds_id'] not in counts_per_ds or counts_per_ds[row['ds_id']] < max_num:

        if len(row['simp'].lower().strip()) > 0 and len(row['src'].lower().strip()) > 0:

            if row['simp'].lower() not in all_txts:

                decision = random.randint(0, 1)

                if (decision == 0 and num_simp_per_ds[row['ds_id']] < max_num/2) or (num_src_per_ds[row['ds_id']] == max_num/2 and num_simp_per_ds[row['ds_id']] < max_num/2):
                    num_simp_per_ds[row['ds_id']] += 1
                    text_and_base_ds.append((row['simp'], row['ds_id'], 'simp'))
                    all_txts.append(row['simp'].lower())
                    only_texts.append(row['simp'])

                    counts_per_ds[row['ds_id']] = counts_per_ds[row['ds_id']] + 1
                elif (decision == 1 and num_src_per_ds[row['ds_id']] < max_num/2) or (num_simp_per_ds[row['ds_id']] == max_num/2 and num_src_per_ds[row['ds_id']] < max_num/2):
                    num_src_per_ds[row['ds_id']] += 1
                    text_and_base_ds.append((row['src'], row['ds_id'], 'src'))
                    all_txts.append(row['src'].lower())
                    only_texts.append(row['src'])
                    
                    counts_per_ds[row['ds_id']] = counts_per_ds[row['ds_id']] + 1

In [5]:
len(only_texts)

1260

In [6]:
len(text_and_base_ds)

1260

In [7]:
with open('/workspace/datasets/ARTS_only_texts.pkl', 'wb') as f:
    pickle.dump(pd.DataFrame(only_texts), f)

In [8]:
with open('/workspace/datasets/ARTS_triples.pkl', 'wb') as f:
    pickle.dump(text_and_base_ds, f)